In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

# 1. Import the cleaned dataset

### 1.1 Read the dataset

In [2]:
data= pd.read_csv('Data/final_dataset_v3.csv')
data.head()

,AMT_CREDIT_SUM,buro_count,CREDIT_ACTIVE_Active,DAYS_CREDIT_UPDATE,DAYS_CREDIT,CREDIT_ACTIVE_Closed,AMT_DOWN_PAYMENT,RATE_DOWN_PAYMENT,CNT_INSTALMENT_FUTURE,SK_DPD_DEF_x,SK_DPD_x,MONTHS_BALANCE_x,DAYS_TERMINATION,DAYS_LAST_DUE_1ST_VERSION,DAYS_FIRST_DUE,DAYS_LAST_DUE,AMT_GOODS_PRICE_y,AMT_ANNUITY,CNT_PAYMENT,NAME_GOODS_CATEGORY_XNA,NAME_PORTFOLIO_POS,NAME_PRODUCT_TYPE_walk-in,PRODUCT_COMBINATION_Cash X-Sell: low,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_PAYMENT_TYPE_XNA,NAME_PAYMENT_TYPE_Cash through the bank,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Approved,NAME_CLIENT_TYPE_New,CODE_REJECT_REASON_XAP,AMT_APPLICATION,NAME_CONTRACT_TYPE_Consumer loans,SELLERPLACE_AREA,DAYS_DECISION,HOUR_APPR_PROCESS_START_y,AMT_CREDIT_y,AMT_PAYMENT,AMT_PAYMENT_x,AMT_PAYMENT_y,DAYS_ENTRY_PAYMENT,DAYS_ENTRY_PAYMENT_x,DAYS_ENTRY_PAYMENT_y,NUM_INSTALMENT_VERSION_x,NUM_INSTALMENT_NUMBER_x,DAYS_INSTALMENT_x,AMT_INSTALMENT_x,DAYS_INSTALMENT,DAYS_INSTALMENT_y,AMT_INSTALMENT,AMT_INSTALMENT_y,EXT_SOURCE_2,AMT_GOODS_PRICE_x,AMT_ANNUITY_x,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,AMT_CREDIT_x,AMT_INCOME_TOTAL,HOUSETYPE_MODE_block of flats,EMERGENCYSTATE_MODE_No,REG_CITY_NOT_WORK_CITY,NAME_INCOME_TYPE_Working,CODE_GENDER_M,CODE_GENDER_F,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,TARGET
0,49655.52,2.0,0.500000,-444.000000,-932.500000,0.500000,9750.00,0.122126,6.190476,0.000000,0.000000,-13.000000,-189.333333,-96.666667,-346.666667,-196.666667,110464.50,13546.455000,9.333333,0.400,0.60,0.000,0.0,0.000,0.40,0.000,0.40,0.60,0.00,0.600,0.200,1.000,66278.70,0.600,570.200,-323.00,15.00,65916.9000,5843.385,24226.086429,91624.095,-830.0,-415.619048,-48.0,2.238095,4.000000,-389.238095,18142.748571,-800.0,-28.0,5843.385,68871.69,0.497963,45000.0,5877.0,-833.0,1.0,2.0,2.0,-2593.0,-2593.0,-1789.0,-10704.0,0.009334,50940.0,112500.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
1,765000.00,5.0,0.600000,-152.000000,-578.600000,0.400000,9000.00,0.151420,2.625000,0.000000,0.000000,-30.500000,-831.000000,-840.000000,-990.000000,-840.000000,135000.00,24183.337500,12.000000,0.750,0.25,0.000,0.0,0.500,0.00,0.250,0.75,0.25,0.00,0.250,0.250,1.000,0.00,0.250,9.000,-344.00,10.75,45000.0000,11323.215,11358.952500,11366.100,-999.0,-921.666667,-845.0,1.000000,3.500000,-915.000000,11358.952500,-990.0,-840.0,11323.215,11366.10,0.285898,900000.0,48442.5,-1021.0,1.0,3.0,3.0,-1753.0,-1190.0,-347.0,-9070.0,0.020246,1006920.0,270000.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
2,1594787.50,9.0,0.444444,-201.555556,-981.444444,0.555556,0.00,0.000000,6.000000,0.000000,0.000000,-6.000000,-158.000000,8.000000,-330.000000,-369.000000,135000.00,2250.000000,12.000000,0.000,1.00,0.000,0.0,0.000,0.00,0.000,0.00,1.00,0.00,1.000,0.000,1.000,0.00,1.000,-1.000,-308.00,13.00,45000.0000,67.500,9425.835000,45000.000,-2919.0,-143.000000,-31.0,1.000000,3.500000,-144.000000,12074.310000,-2919.0,-7.0,1350.000,45000.00,0.617242,450000.0,9000.0,0.0,1.0,2.0,3.0,-529.0,-977.0,-2175.0,-16973.0,0.018029,450000.0,450000.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,40934.25,2.0,0.500000,-957.000000,-1144.000000,0.500000,5770.62,0.058682,8.652174,1.695652,1.695652,-49.760870,145214.400000,72254.200000,-330.000000,145206.400000,94380.48,8071.656429,13.714286,0.125,0.75,0.125,0.0,0.125,0.25,0.375,0.25,0.75,0.25,0.625,0.125,0.625,94380.48,0.875,145.625,-1177.25,14.25,92727.5625,33.165,4239.924911,11042.280,-2364.0,-1440.321429,-810.0,1.053571,11.678571,-1432.321429,5517.468482,-2350.0,-785.0,1350.000,11042.28,0.676706,450000.0,54252.0,0.0,1.0,2.0,2.0,-3426.0,-8821.0,-1603.0,-12093.0,0.035792,450000.0,225000.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
4,1036872.00,10.0,0.600000,-556.700000,-1182.200000,0.300000,7645.50,0.098215,6.000000,0.000000,0.000000,-23.761905,243326.000000,121951.000000,-620.333333,243325.333333,135000.00,28571.062

In [3]:
from pandas_profiling import ProfileReport


# 2. Data Preprocessing

### 2.1 Data Seperation

In [4]:
#Extracting Y value from the dataframe
filtered_columns = [i for i in data.columns if i not in ['TARGET']]
Y= data.TARGET
X = data[filtered_columns]

In [5]:
# Split into train & test with 30% test size
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=1)

In [6]:
# Check the split shapes
print('X_train:',X_train.shape)
print('Y_train:', Y_train.shape)
print('X_test:', X_test.shape)
print('Y_test:', Y_test.shape)

X_train: (395760, 74)
Y_train: (395760,)
X_test: (169612, 74)
Y_test: (169612,)


### 2.2 Scale the test & train datasets

In [7]:
# Scaling the test and train dataframes
num_cols = X_test.columns
scaler_num = StandardScaler()
X_train_num1 = scaler_num.fit_transform(X_train)

X_train_num1 = scaler_num.transform(X_train)
X_test_num1 = scaler_num.transform(X_test)

X_train = pd.DataFrame(X_train_num1, columns=num_cols)
X_test = pd.DataFrame(X_test_num1, columns=num_cols)

# 3. Experimenting Top best performing algorithms

### 3.1 LGBMClassifier

In [ ]:
lgbm = LGBMClassifier(n_estimators=1000, n_jobs=4,
        learning_rate=0.03,
        num_leaves=34,
        colsample_bytree=0.9,
        subsample=0.8,
        max_depth=30,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=375,
        silent=-1,
        verbose=-1,random_state=1
        )
lgbm.fit(X_train,Y_train)
predictions = lgbm.predict_proba(X_test)[:, 1]
auc = metrics.roc_auc_score(Y_test, predictions)
print('AUC:',auc) # AUC: 0.829979058439783

### 3.2 LogisticRegression

In [ ]:
logistic = LogisticRegression(random_state=1, n_jobs=4)
logistic.fit(X_train,Y_train)
predictions = logistic.predict_proba(X_test)[:, 1]
auc = metrics.roc_auc_score(Y_test, predictions)
print('AUC:',auc) # AUC: 0.7428057026278816

### 3.3 Random Forest

In [8]:
forest = ensemble.RandomForestClassifier(random_state=1,max_depth=50, n_jobs=4)
forest.fit(X_train,Y_train)
predicted_y = forest.predict(X_test)
auc = metrics.roc_auc_score(Y_test, predicted_y)
print('AUC:',auc) # AUC: 0.9934888865345095

AUC: 0.9939369655784459


# 4. Building a VotingClassifier

### 4.1 Kfold validation 

In [ ]:
data= X_train
y = Y_train
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds = np.zeros(data.shape[0])


feature_importance_df = pd.DataFrame()

feats = [f for f in data.columns if f not in ['SK_ID_CURR']]

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(data)):
    trn_x, trn_y = data[feats].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = data[feats].iloc[val_idx], y.iloc[val_idx]
    
    voting=ensemble.VotingClassifier(estimators=[('forest', forest),
                                              ('logit', logistic),
                                              ('lgbm', lgbm)], voting='soft', n_jobs=4, flatten_transform=True)
    voting.fit(trn_x, trn_y
           )
    oof_preds[val_idx] = voting.predict(val_x)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, metrics.roc_auc_score(val_y, oof_preds[val_idx])))
    del voting, trn_x, trn_y, val_x, val_y

print('Full AUC score %.6f' % metrics.roc_auc_score(y, oof_preds)) 

### 4.2 Training the model

In [ ]:
# Train the voting classifier
voting =ensemble.VotingClassifier(estimators=[('forest', forest),
                                              ('logit', logistic),
                                              ('lgbm', lgbm)], voting='soft',n_jobs=4, flatten_transform=True)
voting.fit(X_train,Y_train)

In [ ]:
predicted_y = voting.predict(X_test)

### 4.3 Classification Report

In [ ]:
auc = metrics.roc_auc_score(Y_test, predicted_y)
accuracy = metrics.accuracy_score(Y_test, predicted_y)
print(classification_report(Y_test, predicted_y))
print('AUC:',auc)
print('Accuracy:',accuracy)

### 4.4 Confusion Matrix

In [ ]:
cnf_matrix = metrics.confusion_matrix(Y_test, predicted_y)
class_names=['Re-paid(0)','Not Re-paid(1)'] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g', xticklabels=class_names,
    yticklabels=class_names)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.savefig("Confusion_matrix.jpg", dpi=100)

### 4.5 ROC Cuve

In [ ]:
%matplotlib inline
fpr, tpr, thresholds = metrics.roc_curve(Y_test, predicted_y)
auc = metrics.roc_auc_score(Y_test, predicted_y)
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, marker='.')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.title('ROC curve', fontsize = 20)
plt.xlabel('FPR', fontsize=15)
plt.ylabel('TPR', fontsize=15)
plt.grid()
plt.legend(["AUC=%.3f"%auc])
#plt.show()
plt.savefig("Roc_curve.jpg", dpi=100)

### 4.6 Top 20 feature Importances out of 95

In [61]:
features = dict(zip(X_train.columns, forest.feature_importances_))
df=pd.DataFrame(features.items(), columns=['Features','Values'])

df = df.sort_values('Values', ascending=False).reset_index(drop=True)
df.index = df.index + 1
df = df.reset_index().rename(columns={'index': 'Rank'}).set_index('Rank').reset_index()
df.head(20)

,Rank,Features,Values
0,1,EXT_SOURCE_2,0.066786
1,2,DAYS_BIRTH,0.030186
2,3,DAYS_CREDIT,0.027638
3,4,DAYS_CREDIT_UPDATE,0.024560
4,5,DAYS_EMPLOYED,0.023906
5,6,DAYS_ID_PUBLISH,0.023103
6,7,AMT_PAYMENT,0.022877
7,8,DAYS_REGISTRATION,0.021384
8,9,AMT_ANNUITY_x,0.021280
9,10,AMT_CREDIT_x,0.020708


In [62]:
desc=pd.read_csv('Data/HomeCredit_columns_description.csv', encoding='cp1252')
ddf=df.merge(desc,how='inner', left_on='Features', right_on='Row')

In [85]:
final = ddf[['Rank','Features','Description','Values']].set_index('Rank')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_colwidth', 50)

In [86]:
final.head(20)

,Features,Description,Values
Rank,,,
1,EXT_SOURCE_2,Normalized score from external data source,0.066786
2,DAYS_BIRTH,Client's age in days at the time of application,0.030186
3,DAYS_CREDIT,How many days before current application did c...,0.027638
4,DAYS_CREDIT_UPDATE,How many days before loan application did last...,0.024560
5,DAYS_EMPLOYED,How many days before the application the perso...,0.023906
6,DAYS_ID_PUBLISH,How many days before the application did clien...,0.023103
7,AMT_PAYMENT,Goods price of good that client asked for (if ...,0.022877
8,DAYS_REGISTRATION,How many days before the application did clien...,0.021384
9,AMT_ANNUITY_x,Annuity of the Credit Bureau credit,0.021280
